This Python script helps identify district names that are present in one Excel sheet but missing from another, even if there are slight variations in their spelling. It uses fuzzy matching techniques to account for these discrepancies.

-----

### Functions

The script consists of three main functions:

  * **`clean_district_name(name)`**:

      * **Purpose**: This function takes a district name (string) as input and cleans it by:
          * Converting it to lowercase.
          * Removing common prefixes like "Dr.", "Shri", "Saint", "St.", and "District".
          * Eliminating special characters and extra spaces.
      * **Returns**: A cleaned, standardized version of the district name.

  * **`are_districts_similar(dist1, dist2, threshold=80)`**:

      * **Purpose**: This function compares two district names for similarity using various fuzzy matching algorithms from the `fuzzywuzzy` library.
      * **Parameters**:
          * `dist1`, `dist2`: The two district names to compare.
          * `threshold`: An integer (defaulting to 80) representing the minimum similarity score required for two names to be considered a match.
      * **How it works**: It applies several fuzzy matching ratios (ratio, partial ratio, token sort ratio, token set ratio) and returns `True` if any of them meet or exceed the specified `threshold`. It also returns `True` for an exact match after cleaning.
      * **Returns**: `True` if the districts are considered similar, `False` otherwise.

  * **`find_missing_districts(file_path, sheet1_name='Sheet1', sheet2_name='Sheet2')`**:

      * **Purpose**: This is the core function that orchestrates the comparison between two Excel sheets to find missing districts.
      * **Parameters**:
          * `file_path`: The full path to your Excel file.
          * `sheet1_name`: The name of the first sheet (default is 'Sheet1').
          * `sheet2_name`: The name of the second sheet (default is 'Sheet2').
      * **Process**:
        1.  Reads data from both specified sheets into pandas DataFrames.
        2.  Iterates through each row in the **first sheet (`sheet1_name`)**.
        3.  For each district in `sheet1_name`, it filters the districts in `sheet2_name` to only consider those belonging to the **same state**.
        4.  It then uses `are_districts_similar` to check if the `sheet1_name` district has a fuzzy match within the corresponding state's districts in `sheet2_name`.
        5.  If no similar district is found, the district from `sheet1_name` (along with its state) is added to a list of missing districts.
        6.  Finally, it converts the list of missing districts into a pandas DataFrame and saves it as a new Excel file on your **desktop**. The output file name includes a timestamp for uniqueness (e.g., `missing_districts_YYYYMMDD_HHMMSS.xlsx`).
      * **Returns**: A pandas DataFrame containing the list of districts found in `sheet1_name` but not in `sheet2_name`.

-----

### How to Use

1.  **Save the code**: Save the provided code as a Python file (e.g., `district_checker.py`).
2.  **Install libraries**: Ensure you have the necessary libraries installed. If not, open your terminal or command prompt and run:
    ```bash
    pip install pandas openpyxl fuzzywuzzy python-Levenshtein
    ```
    (Note: `python-Levenshtein` is a C implementation that speeds up `fuzzywuzzy`.)
3.  **Prepare your Excel file**: Your Excel file should contain at least two sheets, each with columns named **'State'** and **'District'**.
4.  **Run the script**: Execute the Python script from your terminal:
    ```bash
    python district_checker.py
    ```
5.  **Provide inputs**: The script will prompt you to:
      * Enter the full path to your Excel file.
      * Optionally, enter the names of the first and second sheets you want to compare (press Enter to use the default 'Sheet1' and 'Sheet2').

-----

### Output

The script will print:

  * The total number of missing districts found.
  * The path where the results Excel file has been saved on your desktop.
  * The first few rows of the DataFrame containing the missing districts directly in your console.

A new Excel file (e.g., `missing_districts_20250629_152039.xlsx`) will be created on your desktop, listing the states and districts that were found in the first sheet but not in the second (based on the fuzzy matching logic).

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
from datetime import datetime

def clean_district_name(name):
    """Clean district names by removing common prefixes/suffixes and standardizing format"""
    if not isinstance(name, str):
        return str(name)
    
    # Convert to lowercase for comparison
    name = name.lower()
    
    # Remove common prefixes
    prefixes = ['dr.', 'dr ', 'shri ', 'saint ', 'st.', 'st ', 'district']
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):].strip()
    
    # Remove special characters and extra spaces
    name = ' '.join(name.split())
    
    return name

def are_districts_similar(dist1, dist2, threshold=80):
    """Check if two district names are similar using multiple fuzzy matching approaches"""
    dist1 = clean_district_name(dist1)
    dist2 = clean_district_name(dist2)
    
    # Direct match after cleaning
    if dist1 == dist2:
        return True
    
    # Ratio match
    ratio = fuzz.ratio(dist1, dist2)
    if ratio >= threshold:
        return True
    
    # Partial ratio match (for substring matches)
    partial_ratio = fuzz.partial_ratio(dist1, dist2)
    if partial_ratio >= threshold:
        return True
    
    # Token sort ratio (for word order differences)
    token_sort_ratio = fuzz.token_sort_ratio(dist1, dist2)
    if token_sort_ratio >= threshold:
        return True
    
    # Token set ratio (for handling extra/missing words)
    token_set_ratio = fuzz.token_set_ratio(dist1, dist2)
    if token_set_ratio >= threshold:
        return True
    
    return False

def find_missing_districts(file_path, sheet1_name='Sheet1', sheet2_name='Sheet2'):
    """Find districts present in sheet1 but not in sheet2, accounting for fuzzy matching"""
    
    # Read both sheets
    df1 = pd.read_excel(file_path, sheet_name=sheet1_name)
    df2 = pd.read_excel(file_path, sheet_name=sheet2_name)
    
    # Initialize list to store missing districts
    missing_districts = []
    
    # Iterate through sheet1 districts
    for idx, row1 in df1.iterrows():
        state1 = row1['State']
        district1 = row1['District']
        
        # Get all districts from sheet2 for the same state
        state_districts_sheet2 = df2[df2['State'] == state1]['District'].tolist()
        
        # Check if district exists in sheet2 (using fuzzy matching)
        found_match = False
        for district2 in state_districts_sheet2:
            if are_districts_similar(district1, district2):
                found_match = True
                break
        
        # If no match found, add to missing districts
        if not found_match:
            missing_districts.append({
                'State': state1,
                'District': district1
            })
    
    # Create DataFrame from missing districts
    missing_df = pd.DataFrame(missing_districts)
    
    # Save results to desktop
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_file = os.path.join(desktop_path, f'missing_districts_{timestamp}.xlsx')
    
    missing_df.to_excel(output_file, index=False)
    
    print(f"Found {len(missing_districts)} missing districts")
    print(f"Results saved to: {output_file}")
    
    return missing_df

if __name__ == "__main__":
    # Replace with your Excel file path
    excel_file = input("Enter the path to your Excel file: ")
    
    # Get sheet names from user (optional)
    sheet1 = input("Enter name of first sheet (default: Sheet1): ") or 'Sheet1'
    sheet2 = input("Enter name of second sheet (default: Sheet2): ") or 'Sheet2'
    
    # Run the comparison
    missing_districts = find_missing_districts(excel_file, sheet1, sheet2)
    
    # Display first few missing districts
    print("\nFirst few missing districts:")
    print(missing_districts.head())

This Python script is designed to compare two sheets within an Excel file, specifically looking for district names present in the 'Dealer Master' sheet that are *not* found (or are not sufficiently similar) in the 'RTO' sheet. It leverages fuzzy string matching to account for variations in spelling or formatting of district names.

-----

### Functions

The script consists of three main functions:

  * **`clean_district_name(name)`**:

      * **Purpose**: Standardizes district names for more effective comparison.
      * **Parameters**:
          * `name` (str): The district name to be cleaned.
      * **Process**:
          * Checks if the input is a string; if not, converts it to a string.
          * Converts the name to lowercase.
          * Removes a predefined list of common prefixes (e.g., 'dr.', 'shri ', 'district', 'dist.').
          * Removes any special characters and normalizes multiple spaces into single spaces.
      * **Returns**: A cleaned and standardized `str` representing the district name.

  * **`are_districts_similar(dist1, dist2, threshold=80)`**:

      * **Purpose**: Determines if two district names are similar enough, using multiple fuzzy matching algorithms.
      * **Parameters**:
          * `dist1` (str): The first district name for comparison.
          * `dist2` (str): The second district name for comparison.
          * `threshold` (int, optional): The minimum similarity score (out of 100) required for a match. Defaults to 80.
      * **Process**:
          * First, cleans both `dist1` and `dist2` using `clean_district_name`.
          * Performs an exact match check after cleaning.
          * Applies several `fuzzywuzzy` string similarity ratios:
              * `fuzz.token_sort_ratio`: Good for differences in word order.
              * `fuzz.partial_ratio`: Useful for substring matches.
              * `fuzz.token_set_ratio`: Handles extra or missing words in one string compared to another.
          * If any of these ratios meet or exceed the specified `threshold`, the function considers the districts similar.
      * **Returns**: `True` if the districts are considered similar, `False` otherwise.

  * **`find_missing_districts_and_save(file_path, sheet1_name='RTO', sheet2_name='Dealer Master', score_cutoff=80)`**:

      * **Purpose**: Reads district data from two specified sheets, identifies districts present in the second sheet (`Dealer Master`) but not found (or not similar enough) in the first sheet (`RTO`), and saves these "missing" districts to a new Excel file on the user's desktop.
      * **Parameters**:
          * `file_path` (str): The full path to the Excel file.
          * `sheet1_name` (str, optional): The name of the first sheet to read (e.g., 'RTO'). Defaults to 'RTO'.
          * `sheet2_name` (str, optional): The name of the second sheet to read (e.g., 'Dealer Master'). Defaults to 'Dealer Master'.
          * `score_cutoff` (int, optional): The similarity threshold (0-100) used by `are_districts_similar`. Defaults to 80.
      * **Process**:
        1.  Loads data from `sheet1_name` into `table1` and `sheet2_name` into `table2` using `pandas.read_excel`.
        2.  Initializes an empty list `missing_districts` to store the findings.
        3.  Iterates through each unique 'State' in `table2` (the 'Dealer Master' sheet).
        4.  For each `state_t2`, it extracts all districts associated with that state from both `table1` and `table2`.
        5.  For every `district_t2` in the `Dealer Master` sheet for the current state:
              * It attempts to find a similar district in the 'RTO' sheet's districts for the *same state* using the `are_districts_similar` function.
              * If no sufficiently similar match is found, the `state_t2` and the `district_t2` (from 'Dealer Master') are added to the `missing_districts` list.
        6.  Converts the `missing_districts` list into a pandas DataFrame.
        7.  Constructs a unique filename with a timestamp (e.g., `missing_dealer_master_districts_20250629_152208.xlsx`) and saves the DataFrame to an Excel file on the user's desktop.
        8.  Prints messages indicating the number of missing districts found and the output file path, or if no missing districts were identified.
      * **Returns**: A pandas DataFrame containing the districts found in `sheet2_name` but not in `sheet1_name` (based on fuzzy matching). If no districts are missing, an empty DataFrame is returned.

-----

### How to Use

1.  **Dependencies**: Ensure you have the necessary Python libraries installed. If not, open your terminal or command prompt and run:

    ```bash
    pip install pandas openpyxl fuzzywuzzy python-Levenshtein
    ```

    (`python-Levenshtein` is highly recommended for performance with `fuzzywuzzy`.)

2.  **Excel File Structure**: Your Excel file (`Kill me.xlsx` in the example) must contain at least two sheets, named 'RTO' and 'Dealer Master' (or whatever you specify for `sheet1_name` and `sheet2_name`). Both sheets must have columns titled 'State' and 'District'.

3.  **Update File Path**: Before running, **modify the `excel_file` variable** in the `if __name__ == "__main__":` block to the exact path of your Excel file:

    ```python
    excel_file = r"C:\Users\ASUS\Downloads\Kill me.xlsx" # Make sure this path is correct
    ```

      * **Note**: Using an `r` prefix before the string (raw string) handles backslashes correctly in Windows file paths.

4.  **Adjust Threshold (Optional)**: You can modify the `similarity_threshold` variable (default is 80) to control how strict or lenient the fuzzy matching should be. A higher number (e.g., 90-100) means a closer match is required, while a lower number (e.g., 60-70) will find more loosely related names.

5.  **Run the Script**: Execute the Python script from your terminal:

    ```bash
    python your_script_name.py
    ```

    (Replace `your_script_name.py` with the actual name you saved the file as.)

-----

### Output

Upon execution, the script will:

  * Print messages to the console indicating how many missing districts were found and the full path to the generated output file on your desktop.
  * If missing districts are found, it will also print the head (first few rows) of the DataFrame containing these missing districts directly to the console.
  * An Excel file, named similar to `missing_dealer_master_districts_YYYYMMDD_HHMMSS.xlsx`, will be created in your operating system's desktop directory. This file will contain two columns: 'State' and 'Missing District (from Dealer Master)', listing all the districts identified as missing from the 'RTO' sheet based on the fuzzy matching criteria.